In [1]:
!pip install -U transformers accelerate bitsandbytes peft trl qwen-vl-utils
!pip install -U unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 69.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 19.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.0/557.0 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.7/536.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 8.4 MB/s eta 0:00:0000:01m00:01
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
      Successfully uninstalled huggingface-hub-0.36.0
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0
  Attempting uninstall: transformers
    Found 

In [2]:
import torch
from transformers import Qwen2_5_VLForConditionalGeneration, AutoProcessor, BitsAndBytesConfig
from qwen_vl_utils import process_vision_info
from peft import LoraConfig, get_peft_model

model_id = "Qwen/Qwen2.5-VL-7B-Instruct"

# 1. Cấu hình 4-bit để tiết kiệm VRAM
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# 2. Load Model & Processor
processor = AutoProcessor.from_pretrained(model_id)
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Model đã sẵn sàng!")

2026-01-31 15:19:20.853414: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769872761.045904      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769872761.102778      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769872761.584166      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769872761.584197      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769872761.584200      55 computation_placer.cc:177] computation placer alr

preprocessor_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

model-00005-of-00005.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

Model đã sẵn sàng!


In [3]:
from PIL import Image
import json
from tqdm.auto import tqdm 
import os
import torch
from qwen_vl_utils import process_vision_info

def prepare_m3id_dpo_data(sharegpt_json_path, base_image_dir, num_samples=10):
    with open(sharegpt_json_path, 'r') as f:
        raw_data = json.load(f)

    final_dpo_dataset = []
    
    for item in tqdm(raw_data[:num_samples], desc="Đang tạo DPO samples"):
        try:
            # 1. Xử lý đường dẫn ảnh chính xác như bạn đã mô tả
            rel_path = item['image'].replace('coco/', 'coco2017/') 
            image_path = os.path.join(base_image_dir, rel_path)
            
            if not os.path.exists(image_path):
                continue

            raw_image = Image.open(image_path).convert("RGB")
            
            # Lấy câu hỏi và câu trả lời chuẩn (Chosen)
            question = item['conversations'][0]['value'].replace('<image>', '').strip()
            chosen_answer = item['conversations'][1]['value']
            
            # --- 2. TẠO REJECTED (Blind Inference) ---
            # Quan trọng: Đổi tên biến output thành 'model_inputs' để tránh đè biến 'processor'
            messages_blind = [{"role": "user", "content": [{"type": "text", "text": question}]}]
            
            # Sử dụng processor để format prompt
            text_blind = processor.apply_chat_template(messages_blind, tokenize=False, add_generation_prompt=True)
            
            # Lưu ý: Đặt tên biến là blind_inputs, KHÔNG ĐƯỢC đặt là processor = processor(...)
            blind_inputs = processor(
                text=[text_blind], 
                images=None, 
                padding=True, 
                return_tensors="pt"
            ).to("cuda")
            
            with torch.no_grad():
                # Sinh câu trả lời "mù" (không thấy ảnh)
                generated_ids = model.generate(**blind_inputs, max_new_tokens=128)
                # Cắt bỏ phần prompt trong kết quả trả về
                generated_ids_trimmed = [
                    out_ids[len(in_ids):] for in_ids, out_ids in zip(blind_inputs.input_ids, generated_ids)
                ]
                rejected_answer = processor.batch_decode(
                    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
                )[0]

            # 3. ĐÓNG GÓI THEO FORMAT DPO CỦA TRL
            final_dpo_dataset.append({
                "images": image_path,  # <-- lưu path thay vì PIL
                "prompt": [
                    {"role": "user", "content": [{"type": "image"}, {"type": "text", "text": question}]}
                ],
                "chosen": [
                    {"role": "assistant", "content": [{"type": "text", "text": chosen_answer}]}
                ],
                "rejected": [
                    {"role": "assistant", "content": [{"type": "text", "text": rejected_answer}]}
                ]
            })
        except Exception as e:
            print(f"Lỗi tại item: {e}")
            continue

    return final_dpo_dataset

# Thực thi (Nhớ kiểm tra biến processor đã được load đúng ở đầu file chưa)
train_list = prepare_m3id_dpo_data(
    "/kaggle/input/training-vlm/sharegpt4v_instruct_gpt4-vision_cap100k.json", 
    "/kaggle/input/coco-2017-dataset", 
    num_samples=10 ## Tổng số lượng sample là 102025 mẫu 
)

Đang tạo DPO samples:   0%|          | 0/10 [00:00<?, ?it/s]

In [4]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 47,589,376 || all params: 8,339,756,032 || trainable%: 0.5706


In [5]:
from unsloth import FastVisionModel
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    model_name = "Qwen/Qwen2.5-VL-7B-Instruct",
    load_in_4bit = True, 
    device_map = "auto",
    dtype=torch.float16
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/tmp/ipykernel_55/1049584214.py:1: UserWarning: WARNING: Unsloth should be imported before [transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastVisionModel


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Qwen2_5_Vl patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.563 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [6]:
from peft import LoraConfig

model = FastVisionModel.get_peft_model(
    model,
    r = 16,
    lora_alpha = 32,
    lora_dropout = 0.05,
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    bias = "none",
    use_gradient_checkpointing = True,
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


In [7]:
# from datasets import Dataset
# from PIL import Image

# def dpo_collator(features):
#     images = [Image.open(f["image_path"]).convert("RGB") for f in features]
#     return {
#         "prompt": [f["prompt"] for f in features],
#         "chosen": [f["chosen"] for f in features],
#         "rejected": [f["rejected"] for f in features],
#         "images": [[img] for img in images],
#     }

In [8]:
print(train_list[0])

{'images': '/kaggle/input/coco-2017-dataset/coco2017/train2017/000000000009.jpg', 'prompt': [{'role': 'user', 'content': [{'type': 'image'}, {'type': 'text', 'text': 'What do you see happening in this image?'}]}], 'chosen': [{'role': 'assistant', 'content': [{'type': 'text', 'text': "In the center of the image, a vibrant blue lunch tray holds four containers, each brimming with a variety of food items. The containers, two in pink and two in yellow, are arranged in a 2x2 grid.\n\nIn the top left pink container, a slice of bread rests, lightly spread with butter and sprinkled with a handful of almonds. The bread is cut into a rectangle, and the almonds are scattered across its buttery surface.\n\nAdjacent to it in the top right corner, another pink container houses a mix of fruit. Sliced apples with their fresh white interiors exposed share the space with juicy chunks of pineapple. The colors of the apple slices and pineapple chunks contrast beautifully against the pink container.\n\nBel

In [ ]:
from trl import DPOTrainer, DPOConfig 
from transformers import TrainingArguments
from datasets import Dataset, Image as HFImage

train_dataset = Dataset.from_list(train_list)
train_dataset = train_dataset.cast_column("images", HFImage())

training_args = DPOConfig(
    output_dir = "./qwen25vl-m3id-dpo",
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 8,
    learning_rate = 5e-6,
    num_train_epochs = 1,
    logging_steps = 10,
    save_steps = 200,
    save_total_limit = 2,
    bf16 = False,
    fp16 = True,
    remove_unused_columns = False,
    report_to = "none",
    optim = "paged_adamw_8bit",
    dataset_num_proc = 1,
    
)

trainer = DPOTrainer(
    model = model,
    ref_model = None,
    args = training_args,
    beta = 0.1,
    train_dataset = train_dataset,
    tokenizer = tokenizer,
    # data_collator = dpo_collator,
)
trainer.train()

Extracting prompt in train dataset (num_proc=1):   0%|          | 0/10 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=1):   0%|          | 0/10 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=1):   0%|          | 0/10 [00:00<?, ? examples/s]

In [ ]:
def inference_test(image_path, question):
    # Load model đã được merge LoRA weights nếu cần
    messages = [{"role": "user", "content": [{"type": "image", "image": image_path}, {"type": "text", "text": question}]}]
    
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    image_inputs, _ = process_vision_info(messages)
    inputs = processor(text=[text], images=image_inputs, return_tensors="pt").to("cuda")
    
    with torch.no_grad():
        output = model.generate(**inputs, max_new_tokens=200)
        print("Kết quả sau DPO:", processor.batch_decode(output, skip_special_tokens=True)[0])

# Chạy thử
# inference_test("test_image.jpg", "Mô tả chi tiết các thiết bị trong bếp.")